# Classification of Tumor Samples using EM
* Caitlin Dresibach
* Elizabeth Homan
* Morgan Wall

## Import Libraries and Read in the Data

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import norm,bernoulli
from scipy.optimize import minimize, show_options
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
#from sklearn import mixture
import matplotlib.pyplot as plt
import csv
import math
%matplotlib inline

In [ ]:
#Set path
path = 'C:\\Users\\mkw5c\\Documents\\School- spring 2018\\Machine Learning\\midterm project\\'
data = 'data.csv'
labels = 'labels.csv'
rankings = 'rferankings10.txt'

In [ ]:
df1 = pd.read_csv(path + data, encoding= "ISO-8859-1", low_memory=False)

In [ ]:
labels = pd.read_csv(path + labels, encoding= "ISO-8859-1", low_memory=False)

In [ ]:
ranks = pd.read_table(path + rankings, encoding= "ISO-8859-1", low_memory=False, header = None)

## Preprocessing and Data Exploration

In [ ]:
# add the class labels to the gene expression data
df1["Class"] = labels["Class"]

In [ ]:
df1.head()

In [ ]:
df2 = df1.drop("Unnamed: 0", axis=1)

In [ ]:
df2.head()

In [ ]:
ranks.head()

In [ ]:
# create a list of the top 10 genes based on the RFE results, to use for subsetting the full dataset
ranks2 = ranks[ranks[0] == 1]
ranks2.index.name = 'topgenes'
ranks2.reset_index(inplace=True)
ranks2.head()
top_genes = ranks2['topgenes'].tolist()
top_genes.append(20531) # keep the class column

In [ ]:
# take a subset of the data including all observations, but only the top 10 genes and the class labels

df3 = df2.iloc[:,top_genes]
print(df3.shape)
print(df3.columns)

In [ ]:
# define x and y (features and classes)
x = df3.loc[:,:"gene_2318"]
y = df3.loc[:,"Class"]

In [ ]:
# view a distribution of the class labels
h1 = plt.hist(np.array(df2["Class"]), bins=20,normed=False,histtype='stepfilled',alpha=0.8); 

In [ ]:
#  Create a subset of the full dataset for each tumor type (to be used for selecting a random point
#   from each class)

df_prad = df3.loc[df3["Class"] == "PRAD", :]
df_luad = df3.loc[df3["Class"] == "LUAD", :]
df_brca = df3.loc[df3["Class"] == "BRCA", :]
df_kirc = df3.loc[df3["Class"] == "KIRC", :]
df_coad = df3.loc[df3["Class"] == "COAD", :]

## Expectation Maximization Set Up 


In [ ]:
# Basic Dimensions

n = 801  #number of samples
d = 10   #number of features
k = 5    #number of clusters


### Initialize the Means (mu)

In [ ]:
#choose a random datapoint from each cluster to be the starting mean
mu = []
dfs = [df_prad, df_luad, df_brca, df_kirc, df_coad]
for i in range(5):
    mu_ = dfs[i].iloc[np.random.choice(range(len(dfs[i])), 1, False), :]
    mu.append(mu_)

mu_df = pd.DataFrame(columns = dfs[1].columns)
for i in range(len(mu)):
    mu_df = pd.concat([mu_df, mu[i]])
mu = mu_df.iloc[:, 0:10]
mu

#rename the rows
mu.index = ["PRAD", "LUAD", "BRCA", "KIRC", "COAD"]
mu

### Initialize the Covariance (sigma)

In [ ]:
#initialize the covariance matrices as identity matrices for each cluster with dimensions dxd
sigma = [np.eye(d)] * k

### Initialize the Class Probabilities

In [ ]:
#initialize pi- probability of each class

pi_ = [1./k] * k #equally probable
pi_

### Initialize the Responsibility Matrix

In [ ]:
# initialize the responsibility function as 0
gamma = np.zeros((n, k))
gamma.shape

### Define the Probability Distribution Functions
* PDF is the probability of the class of x given theta.  
* mixPDF = sum over all classes [the probability of the class * distribution of the class]
* responsibility function for each sample = PDFk/mixPDF

In [ ]:
# define the Probability Distribution Function to be used in the E- step 
# for computing the responsibility function. 

PDF = lambda mu, sigma: np.linalg.det(sigma) ** -.5 ** (2*np.pi) ** (-n/2.)\
        * np.exp(-.5 *np.einsum('ij, ij -> i', x - mu, np.dot(np.linalg.inv(sigma), \
        (x-mu).T).T))

## Run the Expectation Maximization

In [ ]:
#initialize a loop

log_likelihoods = []
theta_learning = []
means = []
covar = []
probs = []
threshold = 0.0001
max_iter = 10000
counter = 0
converged = False

while not converged: 
    counter += 1 
    for k_value in range(k):
        
        #-- E- Step
        #---- calculate the responsibility function
        gamma[:, k_value] = pi_[k_value] * PDF(mu.iloc[k_value], sigma[k_value])

        #---- calculate the log likelihood
    log_likelihood = np.sum(np.log(np.sum(gamma, axis = 1)))
    log_likelihoods.append(log_likelihood)

    #---- normalize so that the responsibility matrix is row stochastic
    gamma = (gamma.T /np.sum(gamma, axis = 1)).T

    #---- determine the number of datapoints falling into each distribution
    N_ks = np.sum(gamma, axis = 0)

    for k_value in range(k):
        #-- M- Step
        #---- calculate the new parameters for each Gaussian

        mu.iloc[k_value] = (np.sum(gamma[:, k_value]* x.T, axis = 1).T) / N_ks[k_value]
        x_mu = np.matrix(x- mu.iloc[k_value])

        sigma[k_value] = np.array(np.dot(np.multiply(x_mu.T, gamma[:, k_value]), x_mu)/ N_ks[k_value])

        pi_[k_value] = 1./ n*N_ks[k_value]

    #-- track progress
    theta0 = namedtuple('theta0', ['mu', 'sigma', 'pi_', 'log_likelihood', 'iterations'])
    theta0.mu = mu
    theta0.sigma = sigma
    theta0.pi_ = pi_
    theta0.log_likelihood = log_likelihoods
    theta0.iterations = len(log_likelihoods)
    
    theta_learning.append(theta0)
    means.append(mu)
    covar.append(sigma)
    probs.append(pi_)
    
    
    #-- check for convergence
    if len(log_likelihoods) < 2 : continue
    if np.abs(log_likelihood - log_likelihoods[-2])< threshold: break
    
    # or reached max iterations? 
    converged = counter >= max_iter 
    
params = namedtuple('params', ['mu', 'sigma', 'pi_', 'log_likelihoods', 'num_iters'])
params.mu = mu
params.sigma = sigma
params.pi_ = pi_
params.log_likelihoods = log_likelihoods
params.num_iters = len(log_likelihoods)  
        
print(len(log_likelihoods))
    


### View Resulting Parameters

In [ ]:
print(params.mu.iloc[0:10, :])
print(params.pi_)

## Use the Distributions from EM to Predict the Class Labels

In [ ]:
# define a function that predicts the probability of each class for each observation using the 
# distribution parameters from the EM model
def predict(x, k):
    p = np.linalg.det(sigma[k]) ** - 0.5 * (2 * np.pi) **\
        (-len(x)/2) * np.exp( -0.5 * np.dot(x - mu.iloc[k] , \
        np.dot(np.linalg.inv(sigma[k]) , (x - mu.iloc[k]).T)))
    prob = pi_[k]*p
    return(prob)

In [ ]:
# create a dataframe to store the predicted probabilites for each class
prob_df = pd.DataFrame(index = np.arange(801), columns = ['PRAD', 'LUAD', 'BRCA', 'KIRC', 'COAD'])

In [ ]:
# calculate the probabilities of each class for each observation
prob_df['PRAD'] = predict(x, 0)
prob_df['LUAD'] = predict(x, 1)
prob_df['BRCA'] = predict(x, 2)
prob_df['KIRC'] = predict(x, 3)
prob_df['COAD'] = predict(x, 4)

In [ ]:
prob_df.head()

In [ ]:
# take the sum of all of the calculated class probabilities

for i in range(len(prob_df)):
    prob_df.loc[i, 'sum'] = sum(prob_df.iloc[i, 0:5])
prob_df.head()

In [ ]:
# divide the probability of each class geiven theta by the sum of the probabilities to get the 
# normalized probability

prob_df["PRAD"] = prob_df["PRAD"]/prob_df['sum']
prob_df["LUAD"] = prob_df["LUAD"]/prob_df['sum']
prob_df["BRCA"] = prob_df["BRCA"]/prob_df['sum']
prob_df["KIRC"] = prob_df["KIRC"]/prob_df['sum']
prob_df["COAD"] = prob_df["COAD"]/prob_df['sum']
prob_df.head()

In [ ]:
# predict the class of each observation by finding the class with the highest probability for each obs.
prob_df['pred'] = prob_df.iloc[:, 0:5].idxmax(axis = 1)
print(prob_df['pred'].value_counts())
print(y.value_counts())

In [ ]:
# initialize columns for the ground truth label and an indicator of whether the prediction is correct
prob_df['groundtruth'] = y
prob_df['correct'] = np.nan

In [ ]:
#determine the accuracy

for i in range(801):
    if prob_df.loc[i, 'pred'] == prob_df.loc[i, 'groundtruth']:
        prob_df.loc[i, 'correct'] = 1
    else: 
        prob_df.loc[i, 'correct'] = 0
prob_df.head()

In [ ]:
# find the total number of correct predictions

total_correct = sum(prob_df['correct'])
total_correct

In [ ]:
# calculate the percentage of correct observations

print('Accuracy of Expectation Maximization classifier on test set: {:.2f}'.format(total_correct/len(prob_df)))